<a href="https://colab.research.google.com/github/evpozdniakov/ds_projects/blob/master/hw6/biological_response.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [271]:
import math
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import ensemble
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials
import optuna
import warnings


### Данные

#### Загрузка

In [2]:
bio_data_orig = pd.read_csv('./_train_sem09__1_.zip')

bio_data_orig.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


#### Анализ

Согласно формулировке задания, никакая предварительная обработка данных не требуется.

Посмотрим лишь являются ли наши данные сбалансированными.

In [6]:
m1 = bio_data_orig['Activity'] == 1
print(f'Молекул с положительным биологическим ответом: {m1.sum()}')

m0 = bio_data_orig['Activity'] == 0
print(f'Молекул с отрицательным биологическим ответом: {m0.sum()}')

Молекул с положительным биологическим ответом: 2034
Молекул с отрицательным биологическим ответом: 1717


Данные можно считать сбалансированными.

#### Подготовка сетов

In [11]:
X = bio_data_orig.drop(columns=['Activity'])
y = bio_data_orig['Activity']

Не смотря на то, что данные являются сбалансированными, имеет смысл сделать стратифицированное разбиение.

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(X_train.shape)
print(X_test.shape)

(3000, 1776)
(751, 1776)


### Обучение моделей

#### Логистическая регрессия

Создадим модель лог. регрессии с параметрами по-умолчанию.

In [154]:
log_reg = linear_model.LogisticRegression(
    random_state=42,
)

log_reg.fit(X_train, y_train)

/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

Получили `ConvergenceWarning`. Попробуем задать `max_iter` чтобы модель могла сойтись. Мы намеренно нарушим ограничение в `50` итераций, поскольку мы сейчас создаем базовую модель (с гиперпараметрами по-умолчанию), с которой будем сравнивать другие модели, обученные с помощью оптимизированных гиперпараметров.

In [155]:
log_reg = linear_model.LogisticRegression(
    max_iter=500,
)

log_reg.fit(X_train, y_train)

LogisticRegression(max_iter=500)

Оценим качество модели по метрике F1 и сохраним это значение в `default_log_reg_f1`

In [159]:
y_ = y_test

y_pred = log_reg.predict(X_test)

default_log_reg_f1 = metrics.f1_score(y_, y_pred)

print(f'F₁ score: {default_log_reg_f1}')

F₁ score: 0.7773766546329723


#### Случайный лес

Создадим модель случайного леса с параметрами по-умолчанию.

In [160]:
rf_clf = ensemble.RandomForestClassifier(
    random_state=42
)

rf_clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

Оценим качество модели по метрике F1 и сохраним ее в `default_rand_forest_f1`

In [162]:
y_ = y_test

y_pred = rf_clf.predict(X_test)

default_rand_forest_f1 = metrics.f1_score(y_, y_pred)

print(f'F₁ score: {default_rand_forest_f1}')

F₁ score: 0.8048484848484848


### Предварительные результаты

Мы обучили две модели с параметрами по-умолчанию. Обе модели дали похожие результаты по метрике F1:

- Логистическая регрессия: `0.78`
- Случайный лес: `0.80`

### Оптимизация гиперпараметров

#### GridSearchCV

##### Оптимизируем лог. регрессию

In [226]:
param_grid = [
    {
        'penalty': ['l2', 'none'],
        'solver': ['lbfgs', 'newton-cg', 'sag'],
        'C': [0.01, 0.03, 0.1, 0.3, 1, 3, 8, 13, 20]
    },
    {
        'penalty': ['l1', 'l2'],
        'solver': ['liblinear'],
        'C': [0.01, 0.03, 0.1, 0.3, 1, 3, 8, 13, 20]
    },
    {
        'penalty': ['elasticnet', 'l1', 'l2', 'none'],
        'solver': ['saga'],
        'C': [0.01, 0.03, 0.1, 0.3, 1, 3, 8, 13, 20]
    },
]

grid_search = GridSearchCV(
    estimator=linear_model.LogisticRegression(
        # неизменяемые параметры
        random_state=42,
        # ограничиваем кол-во итераций согласно требованию задания
        max_iter=50,
        class_weight='balanced',
    ),
    scoring='f1',
    param_grid=param_grid,
    cv=5, # кол-во фолдов кросс-валдиации
    n_jobs = -1, # кол-во ядер
)

При обучении моделей мы получим много предупреждений о том, что наша модель не сошлась. Причиной является обучение с большим значением коэфициента `C`, который является обратным шагу градиентного спуска `α`.

In [227]:
grid_search.fit(X_train, y_train)

/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regressi

GridSearchCV(cv=5,
             estimator=LogisticRegression(class_weight='balanced', max_iter=50,
                                          random_state=42),
             n_jobs=-1,
             param_grid=[{'C': [0.01, 0.03, 0.1, 0.3, 1, 3, 8, 13, 20],
                          'penalty': ['l2', 'none'],
                          'solver': ['lbfgs', 'newton-cg', 'sag']},
                         {'C': [0.01, 0.03, 0.1, 0.3, 1, 3, 8, 13, 20],
                          'penalty': ['l1', 'l2'], 'solver': ['liblinear']},
                         {'C': [0.01, 0.03, 0.1, 0.3, 1, 3, 8, 13, 20],
                          'penalty': ['elasticnet', 'l1', 'l2', 'none'],
                          'solver': ['saga']}],
             scoring='f1')

Посмотрим чего удалось добиться методу `GridSearchCV` на модели лог. регрессии. Сохраним результат в `grid_search_log_reg_f1`

In [228]:
print('Наилучшая комбинация гиперпараметров:')
print(grid_search.best_params_)

grid_search_log_reg_f1 = grid_search.score(X_test, y_test)

print(f'Метрика на тестовой выборке: {grid_search_log_reg_f1}')

Наилучшая комбинация гиперпараметров:
{'C': 0.3, 'penalty': 'l1', 'solver': 'saga'}
Метрика на тестовой выборке: 0.7722289890377588


In [229]:
print('Метрика F1 на модели лог. регрессии с параметрами по-умолчанию:')
print(default_log_reg_f1)

Метрика F1 на модели лог. регрессии с параметрами по-умолчанию:
0.7773766546329723


Нам не удалось улучшить метрику, но мы нашли комбинацию параметров, которая позволяет обучить модель за `50` циклов вместо `500` (столько циклов нам понадобилолсь для обучение модели с гиперпараметрами по-умолчанию) и дает почти такой же результат. Чтобы найти оптимальную комбинацию понадобилось `136` секунд.

##### Оптимизируем случайный лес

In [211]:
param_grid = [
    {
        'criterion': ['gini', 'entropy', 'log_loss'],
        'max_depth': list(np.linspace(10, 50, 10, dtype=int)),
        'max_features': [0.01, 0.03, 0.1],
        'min_samples_leaf': list(np.linspace(1, 9, 5, dtype=int)),
        'n_estimators': list(np.linspace(50, 250, 6, dtype=int)),
    }
]

grid_search = GridSearchCV(
    estimator=ensemble.RandomForestClassifier(
        random_state=42,
    ),
    scoring='f1',
    param_grid=param_grid,
    cv=5, # кол-во фолдов кросс-валдиации
    n_jobs = -1, # кол-во ядер
)


In [212]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid=[{'criterion': ['gini', 'entropy', 'log_loss'],
                          'max_depth': [10, 14, 18, 23, 27, 32, 36, 41, 45, 50],
                          'max_features': [0.01, 0.03, 0.1],
                          'min_samples_leaf': [1, 3, 5, 7, 9],
                          'n_estimators': [50, 90, 130, 170, 210, 250]}],
             scoring='f1')

Посмотрим чего удалось добиться методу `GridSearchCV` на модели случайного леса. Сохраним результат в `grid_search_rand_forest_f1`

In [213]:
print('Наилучшая комбинация гиперпараметров:')
print(grid_search.best_params_)

grid_search_rand_forest_f1 = grid_search.score(X_test, y_test)
print(f'Метрика на тестовой выборке: {grid_search_rand_forest_f1}')

Наилучшая комбинация гиперпараметров:
{'criterion': 'gini', 'max_depth': 14, 'max_features': 0.1, 'min_samples_leaf': 1, 'n_estimators': 170}
Метрика на тестовой выборке: 0.8062575210589651


In [215]:
print('Метрика F1 на модели случайного леса с параметрами по-умолчанию:')
print(default_rand_forest_f1)

Метрика F1 на модели случайного леса с параметрами по-умолчанию:
0.8048484848484848


Нам удалось немного улучшить метрику. На поиск лучшей комбинации гиперпараметров мы потратили 55(❗️) минут.

#### RandomizedSearchCV

##### Оптимизируем лог. регрессию

In [233]:
param_distributions = [
    {
        'penalty': ['l2', 'none'],
        'solver': ['lbfgs', 'newton-cg', 'sag'],
        'C': [0.01, 0.03, 0.1, 0.3, 1, 3, 8, 13, 20]
    },
    {
        'penalty': ['l1', 'l2'],
        'solver': ['liblinear'],
        'C': [0.01, 0.03, 0.1, 0.3, 1, 3, 8, 13, 20]
    },
    {
        'penalty': ['elasticnet', 'l1', 'l2', 'none'],
        'solver': ['saga'],
        'C': [0.01, 0.03, 0.1, 0.3, 1, 3, 8, 13, 20]
    },
]

random_search = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(
        # неизменяемые параметры
        random_state=42,
        max_iter=50,
        class_weight='balanced',
    ),
    scoring='f1',
    param_distributions=param_distributions,
    n_iter=20,
    cv=5, # кол-во фолдов кросс-валдиации
    n_jobs = -1, # кол-во ядер
)


In [234]:
random_search.fit(X_train, y_train);

/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:35

Посмотрим чего удалось добиться методу `RandomizedSearchCV` на модели лог. регрессии. Сохраним результат в `rand_search_log_reg_f1`

In [235]:
print('Наилучшая комбинация гиперпараметров:')
print(random_search.best_params_)

rand_search_log_reg_f1 = random_search.score(X_test, y_test)

print('Метрика на тестовой выборке:')
print(rand_search_log_reg_f1)

Наилучшая комбинация гиперпараметров:
{'solver': 'liblinear', 'penalty': 'l2', 'C': 0.03}
Метрика на тестовой выборке:
0.7825030376670716


In [236]:
print('Метрика F1 на модели лог. регрессии с параметрами по-умолчанию:')
print(default_log_reg_f1)

print('Метрика F1 на модели лог. регрессии с параметрами GridSearchCV:')
print(grid_search_log_reg_f1)

Метрика F1 на модели лог. регрессии с параметрами по-умолчанию:
0.7773766546329723
Метрика F1 на модели лог. регрессии с параметрами GridSearchCV:
0.7722289890377588


Методу `RandomizedSearchCV` удалось улучшить метрику, причем он отработал в четыре раза быстрее, чем `GridSearchCV` (`37` секунд против `136`).

##### Оптимизируем случайный лес

In [208]:
param_distributions = [
    {
        'criterion': ['gini', 'entropy', 'log_loss'],
        'max_depth': list(np.linspace(10, 50, 10, dtype=int)),
        'max_features': [0.01, 0.03, 0.1],
        'min_samples_leaf': list(np.linspace(1, 9, 5, dtype=int)),
        'n_estimators': list(np.linspace(50, 250, 6, dtype=int)),
    }
]

random_search = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(
        random_state=42,
    ),
    scoring='f1',
    param_distributions=param_distributions,
    n_iter=50,
    cv=5, # кол-во фолдов кросс-валдиации
    n_jobs = -1, # кол-во ядер
)

random_search.fit(X_train, y_train)


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
                   n_iter=50, n_jobs=-1,
                   param_distributions=[{'criterion': ['gini', 'entropy',
                                                       'log_loss'],
                                         'max_depth': [10, 14, 18, 23, 27, 32,
                                                       36, 41, 45, 50],
                                         'max_features': [0.01, 0.03, 0.1],
                                         'min_samples_leaf': [1, 3, 5, 7, 9],
                                         'n_estimators': [50, 90, 130, 170, 210,
                                                          250]}],
                   scoring='f1')

Посмотрим чего удалось добиться методу `RandomizedSearchCV` на модели случайного леса. Сохраним результат в `rand_search_rand_forest_f1`

In [209]:
print('Наилучшая комбинация гиперпараметров:')
print(random_search.best_params_)

rand_search_rand_forest_f1 = random_search.score(X_test, y_test)

print('Метрика на тестовой выборке:')
print(rand_search_rand_forest_f1)

Наилучшая комбинация гиперпараметров:
{'n_estimators': 90, 'min_samples_leaf': 1, 'max_features': 0.1, 'max_depth': 14, 'criterion': 'gini'}
Метрика на тестовой выборке:
0.8076923076923076


In [237]:
print('Метрика F1 на модели случайного леса с параметрами по-умолчанию:')
print(default_rand_forest_f1)

print('Метрика F1 на модели случайного леса с параметрами GridSearchCV:')
print(grid_search_rand_forest_f1)

Метрика F1 на модели случайного леса с параметрами по-умолчанию:
0.8048484848484848
Метрика F1 на модели случайного леса с параметрами GridSearchCV:
0.8062575210589651


Методу `RandomizedSearchCV` удалось найти комбинацию гиперпараметров лучшую, чем нашел `GridSearchCV`, причем он сделал это в 50 раз быстрее!

### Продвинутая оптимизация гиперпараметров

#### Hyperopt

##### Оптимизируем лог. регрессию

In [263]:
space = hp.choice('classifier_type', [
    {
        'penalty': hp.choice('penalty1', ['l2', 'none']),
        'solver': hp.choice('solver1', ['lbfgs', 'newton-cg', 'sag']),
        'C': hp.loguniform('C1', -3, 3),
    },
    {
        'penalty': hp.choice('penalty2', ['l1', 'l2']),
        'solver': hp.choice('solver2', ['liblinear']),
        'C': hp.loguniform('C2', -3, 3),
    },
    {
        'penalty': hp.choice('penalty3', ['elasticnet', 'l1', 'l2', 'none']),
        'l1_ratio': hp.uniform('l1_ratio3', 0.1, 0.9),
        'solver': hp.choice('solver3', ['saga']),
        'C': hp.loguniform('C3', -3, 3),
    },
])

# Нам нужно создать функцию для минимизации. Она должна принимать
# словарь значений гиперпараметров и возвращать значение целевой функции.
random_state = 42
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в `params`
    res_params = {
        'penalty': params['penalty'],
        'solver': params['solver'],
        'max_iter': 50,
        'class_weight': 'balanced',
    }

    if res_params['penalty'] != 'none':
        res_params['C'] = float(params['C'])

    if res_params['penalty'] == 'elasticnet':
        res_params['l1_ratio'] = float(params['l1_ratio'])

    # используем эту комбинацию для построения модели
    model = linear_model.LogisticRegression(**res_params, random_state=random_state)

    # обучаем модель с помощью кросс-валидации
    score = cross_val_score(model, X, y, cv=cv, scoring='f1', n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

# начинаем подбор гиперпараметров
trials = Trials() # используется для логирования результатов

best=fmin(
    hyperopt_rf, # наша функция 
    space=space, # пространство гиперпараметров
    algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
    max_evals=25, # максимальное количество итераций
    trials=trials, # логирование результатов
    rstate=np.random.default_rng(random_state) # фиксируем для повторяемости результата
)


  4%|▍         | 1/25 [00:01<00:29,  1.23s/trial, best loss: -0.7700011865817935]

/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


  8%|▊         | 2/25 [00:03<00:37,  1.62s/trial, best loss: -0.7700011865817935]

/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


 12%|█▏        | 3/25 [00:03<00:25,  1.14s/trial, best loss: -0.7700011865817935]

...

 68%|██████▊   | 17/25 [00:32<00:13,  1.65s/trial, best loss: -0.7835906775926528]

/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


 76%|███████▌  | 19/25 [00:33<00:07,  1.30s/trial, best loss: -0.7835906775926528]

/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


100%|██████████| 25/25 [00:40<00:00,  1.61s/trial, best loss: -0.7835906775926528]


Посмотрим чего удалось добиться методу `Hyperopt` на модели лог. регрессии. Сохраним результат в `hyperopt_log_reg_f1`

In [264]:
print('Наилучшие значения гиперпараметров')
print(hyperopt.space_eval(space, best))

Наилучшие значения гиперпараметров
{'C': 0.17787089549850982, 'penalty': 'l1', 'solver': 'liblinear'}


In [265]:
best_params = hyperopt.space_eval(space, best)

log_reg = linear_model.LogisticRegression(
    penalty=best_params['penalty'],
    C=best_params['C'],
    random_state=42,
    max_iter=50,
    solver=best_params['solver'],
    class_weight='balanced',
)

log_reg.fit(X_train, y_train)

y_test_pred = log_reg.predict(X_test)

hyperopt_log_reg_f1 = metrics.f1_score(y_test, y_test_pred)

print('Метрика на тестовой выборке:')
print(hyperopt_log_reg_f1)

Метрика на тестовой выборке:
0.7762998790810156


In [266]:
print('Метрика F1 на модели лог. регрессии с параметрами по-умолчанию:')
print(default_log_reg_f1)

print('Метрика F1 на модели лог. регрессии с параметрами GridSearchCV:')
print(grid_search_log_reg_f1)

print('Метрика F1 на модели лог. регрессии с параметрами RandomizedSearchCV:')
print(rand_search_log_reg_f1)

Метрика F1 на модели лог. регрессии с параметрами по-умолчанию:
0.7773766546329723
Метрика F1 на модели лог. регрессии с параметрами GridSearchCV:
0.7722289890377588
Метрика F1 на модели лог. регрессии с параметрами RandomizedSearchCV:
0.7825030376670716


Методу `Hyperopt` удалось за `20` секунд подобрать гиперпараметры, которые дают хороший результат (лучше чему у `GridSearchCV`, немного хуже чем у `RandomizedSearchCV`).

##### Оптимизируем случайный лес

In [267]:
space={
    'criterion': hp.choice('criterion', ['gini', 'entropy', 'log_loss']),
    'max_depth' : hp.quniform('max_depth', 10, 50, 1),
    'max_features' : hp.loguniform('max_features', math.log(0.01), math.log(0.1)),
    'min_samples_leaf' : hp.quniform('min_samples_leaf', 1, 9, 1),
    'n_estimators': hp.quniform('n_estimators', 50, 250, 1),
}

# Нам нужно создать функцию для минимизации. Она должна принимать
# словарь значений гиперпараметров и возвращать значение целевой функции.
random_state = 42
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в `params`
    res_params = {
        'criterion': params['criterion'],
        'max_depth': int(params['max_depth']),
        'max_features': float(params['max_features']),
        'min_samples_leaf': int(params['min_samples_leaf']),
        'n_estimators': int(params['n_estimators']),
    }

    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**res_params, random_state=random_state)

    # обучаем модель с помощью кросс-валидации
    score = cross_val_score(model, X, y, cv=cv, scoring='f1', n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

# начинаем подбор гиперпараметров
trials = Trials() # используется для логирования результатов

best=fmin(
    hyperopt_rf, # наша функция 
    space=space, # пространство гиперпараметров
    algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
    max_evals=50, # максимальное количество итераций
    trials=trials, # логирование результатов
    rstate=np.random.default_rng(random_state) # фиксируем для повторяемости результата
)
print('Наилучшие значения гиперпараметров')
print('{}'.format(hyperopt.space_eval(space, best)))


100%|██████████| 50/50 [01:44<00:00,  2.08s/trial, best loss: -0.8260747565080859]
Наилучшие значения гиперпараметров
{'criterion': 'log_loss', 'max_depth': 47.0, 'max_features': 0.0958199712678102, 'min_samples_leaf': 1.0, 'n_estimators': 127.0}


Посмотрим чего удалось добиться методу `Hyperopt` на модели случайного леса. Сохраним результат в `hyperopt_rand_forest_f1`

In [268]:
print('Наилучшие значения гиперпараметров')
print(hyperopt.space_eval(space, best))

Наилучшие значения гиперпараметров
{'criterion': 'log_loss', 'max_depth': 47.0, 'max_features': 0.0958199712678102, 'min_samples_leaf': 1.0, 'n_estimators': 127.0}


In [269]:
best_params = hyperopt.space_eval(space, best)

best_rf_clf = ensemble.RandomForestClassifier(
    criterion=best_params['criterion'],
    max_depth=int(best_params['max_depth']),
    max_features=float(best_params['max_features']),
    min_samples_leaf=int(best_params['min_samples_leaf']),
    n_estimators=int(best_params['n_estimators']),
)

best_rf_clf.fit(X_train, y_train)

y_test_pred = best_rf_clf.predict(X_test)

hyperopt_rand_forest_f1 = metrics.f1_score(y_test, y_test_pred)

print('Метрика на тестовой выборке:')
print(hyperopt_rand_forest_f1)

Метрика на тестовой выборке:
0.8100961538461539


In [270]:
print('Метрика F1 на модели случайного леса с параметрами по-умолчанию:')
print(default_rand_forest_f1)

print('Метрика F1 на модели случайного леса с параметрами GridSearchCV:')
print(grid_search_rand_forest_f1)

print('Метрика F1 на модели случайного леса с параметрами RandomizedSearchCV:')
print(rand_search_rand_forest_f1)

Метрика F1 на модели случайного леса с параметрами по-умолчанию:
0.8048484848484848
Метрика F1 на модели случайного леса с параметрами GridSearchCV:
0.8062575210589651
Метрика F1 на модели случайного леса с параметрами RandomizedSearchCV:
0.8076923076923076


Метод `hyperopt` смог найти гиперпараметры, которые улучшают метрику `F1`. Он потратил на их поиск `104` секунды.

#### Optuna

##### Оптимизируем лог. регрессию

In [296]:
def calc_log_reg_params(all_params):
    solver = all_params['solver']
    penalty_saga = all_params['penalty_saga']
    penalty_liblinear = all_params['penalty_liblinear']
    penalty_others = all_params['penalty_others']
    l1_ratio = all_params['l1_ratio']
    C = all_params['C']

    res_params = {
        'random_state': 42,
        'max_iter': 50,
        'solver': solver,
        'class_weight': 'balanced',
    }

    if solver == 'saga':
        res_params['penalty'] = penalty_saga
    elif solver == 'liblinear':
        res_params['penalty'] = penalty_liblinear
    else:
        res_params['penalty'] = penalty_others

    if solver != 'none':
        res_params['C'] = C

    if res_params['penalty'] == 'elasticnet':
        res_params['l1_ratio'] = l1_ratio

    return res_params

def optuna_rf_log_reg(trial):
    # задаем пространство поиска
    solver = trial.suggest_categorical('solver', ['lbfgs', 'newton-cg', 'sag', 'saga', 'liblinear'])
    penalty_saga = trial.suggest_categorical('penalty_saga', ['elasticnet', 'l1', 'l2', 'none'])
    penalty_liblinear = trial.suggest_categorical('penalty_liblinear', ['l1', 'l2'])
    penalty_others = trial.suggest_categorical('penalty_others', ['l2', 'none'])
    C = trial.suggest_float('C', 0.01, 20, log=True)
    l1_ratio = trial.suggest_float('l1_ratio', 0.1, 0.9)

    # динамически вычисляем параметры
    res_params = calc_log_reg_params({
        'solver': solver,
        'penalty_saga': penalty_saga,
        'penalty_liblinear': penalty_liblinear,
        'penalty_others': penalty_others,
        'C': C,
        'l1_ratio': l1_ratio,
    })

    # создае модель
    log_reg = linear_model.LogisticRegression(**res_params)

    # обучаем модель с помощью кросс-валидации
    score = cross_val_score(log_reg, X_train, y_train, cv=5, scoring='f1', n_jobs=-1).mean()

    return score

Для начала запустим поиск на `15` итерациях.

In [297]:
study = optuna.create_study(study_name="Optuna: Logistic regression", direction="maximize")

study.optimize(optuna_rf_log_reg, n_trials=15)

[I 2023-02-03 21:31:15,689] A new study created in memory with name: Optuna: Logistic regression
[I 2023-02-03 21:31:18,264] Trial 0 finished with value: 0.7587157356835712 and parameters: {'solver': 'liblinear', 'penalty_saga': 'l1', 'penalty_liblinear': 'l2', 'penalty_others': 'l2', 'C': 8.716212145595454, 'l1_ratio': 0.8881358686348735}. Best is trial 0 with value: 0.7587157356835712.
/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/

Продолжим поиск еще на `10` итерациях чтобы в сумме получилось `25` — столько было у `Hyperopt`.

In [298]:
study.optimize(optuna_rf_log_reg, n_trials=10)


/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/ev/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/ev/miniconda3/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:3

Посмотрим чего удалось добиться методу `Optuna` на модели лог. регрессии. Сохраним результат в `optuna_log_reg_f1`

In [295]:
print('Наилучшие значения гиперпараметров')
print(study.best_params)

Наилучшие значения гиперпараметров
{'solver': 'sag', 'penalty_saga': 'l2', 'penalty_liblinear': 'l1', 'penalty_others': 'l2', 'C': 0.04059230752336201, 'l1_ratio': 0.29060987800714877}


In [300]:
res_params = calc_log_reg_params(study.best_params)

log_reg = linear_model.LogisticRegression(**res_params)

log_reg.fit(X_train, y_train)

y_test_pred = log_reg.predict(X_test)

optuna_log_reg_f1 = metrics.f1_score(y_test, y_test_pred)

print('Метрика на тестовой выборке:')
print(optuna_log_reg_f1)

Метрика на тестовой выборке:
0.7868453105968332


In [301]:
print('Метрика F1 на модели лог. регрессии с параметрами по-умолчанию:')
print(default_log_reg_f1)

print('Метрика F1 на модели лог. регрессии с параметрами GridSearchCV:')
print(grid_search_log_reg_f1)

print('Метрика F1 на модели лог. регрессии с параметрами RandomizedSearchCV:')
print(rand_search_log_reg_f1)

print('Метрика F1 на модели лог. регрессии с параметрами Hyperopt:')
print(hyperopt_log_reg_f1)

Метрика F1 на модели лог. регрессии с параметрами по-умолчанию:
0.7773766546329723
Метрика F1 на модели лог. регрессии с параметрами GridSearchCV:
0.7722289890377588
Метрика F1 на модели лог. регрессии с параметрами RandomizedSearchCV:
0.7825030376670716
Метрика F1 на модели лог. регрессии с параметрами Hyperopt:
0.7762998790810156


Методу `Optuna` удалось найти наилучшую комбинацию гиперпараметров лог. регрессии за `25` итераций. Поиск занял меньше одной минуты.

##### Оптимизируем случайный лес

In [304]:
def optuna_rf_rand_forest(trial):
    # задаем пространство поиска
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy', 'log_loss'])
    max_depth = trial.suggest_int('max_depth', 10, 50)
    max_features = trial.suggest_float('max_features', 0.01, 0.1, log=True)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 9)
    n_estimators = trial.suggest_int('n_estimators', 50, 250)

    # создае модель
    rf_clf = ensemble.RandomForestClassifier(
        n_estimators=n_estimators,
        criterion=criterion,
        max_depth=max_depth,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        random_state=42
    )

    # обучаем модель с помощью кросс-валидации
    score = cross_val_score(rf_clf, X_train, y_train, cv=5, scoring='f1', n_jobs=-1).mean()

    return score

Для начала запустим поиск на `15` итерациях.

In [305]:
# создаем процесс обучения и запускаем его
study_rand_forest = optuna.create_study(study_name='Optuna: Random forest', direction='maximize')

study_rand_forest.optimize(optuna_rf_rand_forest, n_trials=15)

[I 2023-02-03 21:52:38,374] A new study created in memory with name: Optuna: Random forest
[I 2023-02-03 21:52:40,920] Trial 0 finished with value: 0.8068634629074264 and parameters: {'criterion': 'entropy', 'max_depth': 50, 'max_features': 0.010817542086391196, 'min_samples_leaf': 4, 'n_estimators': 91}. Best is trial 0 with value: 0.8068634629074264.
[I 2023-02-03 21:52:42,456] Trial 1 finished with value: 0.8103831212490199 and parameters: {'criterion': 'log_loss', 'max_depth': 43, 'max_features': 0.010820588087266022, 'min_samples_leaf': 2, 'n_estimators': 189}. Best is trial 1 with value: 0.8103831212490199.
[I 2023-02-03 21:52:46,798] Trial 2 finished with value: 0.8183537580470472 and parameters: {'criterion': 'entropy', 'max_depth': 45, 'max_features': 0.09132390977954467, 'min_samples_leaf': 5, 'n_estimators': 184}. Best is trial 2 with value: 0.8183537580470472.
[I 2023-02-03 21:52:48,462] Trial 3 finished with value: 0.8165386474243344 and parameters: {'criterion': 'entropy'

Продолжим поиск еще на `10` итерациях чтобы в сумме получилось `25` — столько было у `Hyperopt`.

In [306]:
study_rand_forest.optimize(optuna_rf_rand_forest, n_trials=10)

[I 2023-02-03 21:53:50,406] Trial 15 finished with value: 0.8204763057437813 and parameters: {'criterion': 'gini', 'max_depth': 19, 'max_features': 0.06696054176226654, 'min_samples_leaf': 2, 'n_estimators': 209}. Best is trial 7 with value: 0.8222721625888303.
[I 2023-02-03 21:53:52,805] Trial 16 finished with value: 0.8169510166685587 and parameters: {'criterion': 'gini', 'max_depth': 36, 'max_features': 0.06872338428436221, 'min_samples_leaf': 1, 'n_estimators': 111}. Best is trial 7 with value: 0.8222721625888303.
[I 2023-02-03 21:53:54,571] Trial 17 finished with value: 0.8184182001325052 and parameters: {'criterion': 'gini', 'max_depth': 13, 'max_features': 0.04209989101531248, 'min_samples_leaf': 3, 'n_estimators': 151}. Best is trial 7 with value: 0.8222721625888303.
[I 2023-02-03 21:53:56,427] Trial 18 finished with value: 0.8154505034010834 and parameters: {'criterion': 'gini', 'max_depth': 30, 'max_features': 0.03208798897198854, 'min_samples_leaf': 2, 'n_estimators': 168}. 

Посмотрим чего удалось добиться методу `Optuna` на модели случайного леса. Сохраним результат в `optuna_rand_forest_f1`

In [307]:
print('Наилучшие значения гиперпараметров')
print(study_rand_forest.best_params)

Наилучшие значения гиперпараметров
{'criterion': 'gini', 'max_depth': 17, 'max_features': 0.06526211237838539, 'min_samples_leaf': 2, 'n_estimators': 207}


In [308]:
rf_clf = ensemble.RandomForestClassifier(
    n_estimators=study_rand_forest.best_params['n_estimators'],
    criterion=study_rand_forest.best_params['criterion'],
    max_depth=study_rand_forest.best_params['max_depth'],
    min_samples_leaf=study_rand_forest.best_params['min_samples_leaf'],
    max_features=study_rand_forest.best_params['max_features'],
    random_state=42
)

rf_clf.fit(X_train, y_train)

y_test_pred = rf_clf.predict(X_test)

optuna_rand_forest_f1 = metrics.f1_score(y_test, y_test_pred)

print('Метрика на тестовой выборке:')
print(optuna_rand_forest_f1)


Метрика на тестовой выборке:
0.8110709987966305


In [309]:
print('Метрика F1 на модели случайного леса с параметрами по-умолчанию:')
print(default_rand_forest_f1)

print('Метрика F1 на модели случайного леса с параметрами GridSearchCV:')
print(grid_search_rand_forest_f1)

print('Метрика F1 на модели случайного леса с параметрами RandomizedSearchCV:')
print(rand_search_rand_forest_f1)

print('Метрика F1 на модели случайного леса с параметрами Hyperopt:')
print(hyperopt_rand_forest_f1)

Метрика F1 на модели случайного леса с параметрами по-умолчанию:
0.8048484848484848
Метрика F1 на модели случайного леса с параметрами GridSearchCV:
0.8062575210589651
Метрика F1 на модели случайного леса с параметрами RandomizedSearchCV:
0.8076923076923076
Метрика F1 на модели случайного леса с параметрами Hyperopt:
0.8100961538461539


Методу `Optuna` и здесь удалось найти налилучшую комбинацию гиперпараметров за 25 итераций. На это было потрачено `70` секунд.

### Вывод

Мы сравнили работу четырех алгоритмов для оптимизации гиперпараметров моделей МО:
- базовые `GridSearchCV`, `RandomizedSearchCV`
- и продвинутые `Hyperopt`, `Optuna`

Базовые алгоритмы просты в использовании, но из двух лучше выбирать `RandomizedSearchCV`, потому что он менее требователен к ресурсам.

Из продвинутых алгоритмов `Optuna` выглядит более привлекательным: быстрее работает, проще в настройке, позволяет запускать дополнительные итерации, имеет встроенные методы визуализации.